In [4]:
!pip install --upgrade azureml-sdk azureml-widgets

^C


In [1]:
import azureml.core
from azureml.core import Workspace
from azureml.core import Dataset

# Load the workspace from the saved config file
ws = Workspace.from_config('config.json')
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.19.0 to work with dp100-WS


In [2]:
# Get the default datastore
default_ds = ws.get_default_datastore()

# Enumerate all datastores, indicating which is the default
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

azureml_globaldatasets - Default = False
workspaceblobstore - Default = True
workspacefilestore - Default = False


In [13]:
# upload file to default datastore.  Note that you will need to run this from local server if the files are local
def upload(input_folder_path, file_name):
    #Upload file
    default_ds.upload_files(files=[input_folder_path+file_name], # Upload
                       target_path=output_path, # Put it in a folder path in the datastore
                       overwrite=True, # Replace existing files of the same name
                       show_progress=True)

# transform the file into a tabular data set
def create_tab_dataset(output_path, file_name):
    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, output_path+file_name))

    # Display the first 20 rows as a Pandas dataframe
    tab_data_set.take(20).to_pandas_dataframe()

    return tab_data_set

# register the data set in the workspace for easy consumption
def register_tab_dataset(tab_data_set, name):
    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                            name=name,
                                            description='',
                                            tags = {'format':'CSV'},
                                            create_new_version=True)
    except Exception as ex:
        print(ex)
    print('Datasets registered')

# upload and register data set
def upload_and_register_tabular_dataset(input_folder_path, file_name, output_path, name):
    upload(input_folder_path, file_name)
    tab_data_set = create_tab_dataset(output_path, file_name)
    register_tab_dataset(tab_data_set, name)

Datasets registered


In [ ]:
upload_and_register_tabular_dataset('C:/Users/kelvi/OneDrive/Documents/Data Projects/Data/', 'bureau_balance.csv', 'credit data/', 'bureau balance')
#unknown error when trying with some files.  Only thing I can see that might be the issue is that it happens with large files.  Seems ok with 160MB but anything larger and it fails.  Might be something else though
#workaround just now is to use the azure storage explorer to upload all the files and then register the data sets using these functions

In [ ]:
tab_data_set = create_tab_dataset('credit data/', 'creditcard.csv')
register_tab_dataset(tab_data_set, 'credit card')